In [1]:
from azureml.core import Workspace, Datastore, Experiment, Dataset

c:\Users\Vikas.Yetintala\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
ws=Workspace.from_config("./config")

In [3]:
ws=Workspace.from_config("./config")
az_store= Datastore.get(ws,"azure_sdk_blob01")
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
az_default_store= ws.get_default_datastore()

# creating custom environment

In [4]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

# Hyper parameter Tuning

In [22]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret','opencensus-ext-azure>=1.0.1'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

{
    "assetId": "azureml://locations/eastus/workspaces/3e3bfef7-2a2b-41b0-9796-e1cdf285aece/environments/MyEnvironment/versions/11",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221010.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "MyEnvironment",
    "py

In [23]:
from azureml.core.compute import AmlCompute 
cluster_name= "comput1"
if cluster_name not in ws.compute_targets:
    compute_config= AmlCompute.provisioning_configuration(vm_size="",
                                                        max_nodes=2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster= ws.compute_targets[cluster_name]
    print(cluster," compute cluster found.. using it....")

AmlCompute(workspace=Workspace.create(name='aml-msft-ml-poc1', subscription_id='a4df42c9-c1d2-44c4-b2c2-05edb4b27aad', resource_group='RG-MSFT-ML-POC-01'), name=comput1, id=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.MachineLearningServices/workspaces/aml-msft-ml-poc1/computes/comput1, type=AmlCompute, provisioning_state=Succeeded, location=eastus, tags={})  compute cluster found.. using it....


In [27]:
from azureml.core import ScriptRunConfig 
script_config = ScriptRunConfig(source_directory="Training_python_scripts/",
                                script="hyperdrivescript.py",
                                environment=myenv,
                                compute_target=cluster)

In [28]:
from azureml.train.hyperdrive import GridParameterSampling,choice
hyper_params= GridParameterSampling(
                                    {'--n_estimators': choice(10,20),
                                    '--min_samples_leaf': choice(1,2,)})

In [29]:
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
hyper_config = HyperDriveConfig(run_config=script_config,
                                hyperparameter_sampling=hyper_params,
                                policy=None,
                                primary_metric_name='accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=1)
from azureml.core.experiment import Experiment
new_exp= Experiment(ws,"azureml_titanic_vikas_app_insights")
print("submitting the exxperiment")
new_run= new_exp.submit(hyper_config)
new_run.wait_for_completion(show_output=True)


submitting the exxperiment
RunId: HD_9677629b-1104-4541-af46-83e7653fc746
Web View: https://ml.azure.com/runs/HD_9677629b-1104-4541-af46-83e7653fc746?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourcegroups/RG-MSFT-ML-POC-01/workspaces/aml-msft-ml-poc1&tid=3882b70d-a91e-468c-9928-820358bfbd73

Streaming azureml-logs/hyperdrive.txt

[2022-11-07T08:00:59.147523][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2022-11-07T08:00:59.837767][GENERATOR][INFO]Successfully sampled '1' jobs, they will soon be submitted to the execution target.
[2022-11-07T08:00:59.8745843Z][SCHEDULER][INFO]Scheduling job, id='HD_9677629b-1104-4541-af46-83e7653fc746_0' 
[2022-11-07T08:01:00.3268558Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_9677629b-1104-4541-af46-83e7653fc746_0' 
[2022-11-07T08:02:29.252457][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2022-11-07T08:02:29.6019093Z][SCHEDULER][INFO]Scheduling job, id='HD_9677629b-1104-

{'runId': 'HD_9677629b-1104-4541-af46-83e7653fc746',
 'target': 'comput1',
 'status': 'Completed',
 'startTimeUtc': '2022-11-07T08:00:58.408061Z',
 'endTimeUtc': '2022-11-07T08:06:58.658215Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6d2e803e-b23a-4db8-b6b9-f5f1033189e8',
  'user_agent': 'python/3.9.13 (Windows-10-10.0.22000-SP0) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',
   'amlClientModule': '[Scrubbed]',
   'amlClientFunction': '[Scrubbed]',
   'tenantId': '3882b70d-a91e-468c-9928-820358bfbd73',
   'amlClientRequestId': '1fdc93a7-97e2-40fb-bbfe-187edb95a242',
   'amlClientSessio

In [18]:
print(new_run)

Run(Experiment: azureml_titanic_vikas,
Id: HD_97e77ed6-efbd-4582-a891-6c0ebd5ab413,
Type: hyperdrive,
Status: Completed)
